In [1]:
import pandas as pd
import pyodbc
import numpy as np
import os

In [22]:
path = r'T:\Tableau\tableauJourneyToWork\Datasources'

In [23]:
# adjust accordingly on the tables ModeShare_ALL_Years; ModeByVehiclesAvailable_AllYears
data = pd.read_csv(path + '\\ModeByVehiclesAvailable_AllYears.csv')   
df = pd.DataFrame(data)

print(df)

      Year              Geography  \
0     2009                 Coburg   
1     2009                 Coburg   
2     2009                 Coburg   
3     2009                 Coburg   
4     2009                 Coburg   
...    ...                    ...   
1495  2021  Eugene Urbanized Area   
1496  2021   Salem Urbanized Area   
1497  2021   Salem Urbanized Area   
1498  2021   Salem Urbanized Area   
1499  2021   Salem Urbanized Area   

                      Mode.by.Vehicle.Availability  \
0                 Car, truck, or van - drove alone   
1                   Car, truck, or van - carpooled   
2        Public transportation (excluding taxicab)   
3                                           Walked   
4     Taxicab, motorcycle, bicycle, or other means   
...                                            ...   
1495  Taxicab, motorcycle, bicycle, or other means   
1496  Taxicab, motorcycle, bicycle, or other means   
1497  Taxicab, motorcycle, bicycle, or other means   
1498  Taxicab, 

In [24]:
df.dtypes

Year                              int64
Geography                        object
Mode.by.Vehicle.Availability     object
Vehicles.Available               object
Estimate                          int64
MOE_Est                           int64
Share                           float64
MOE_Share                       float64
SharePCT                        float64
MOE_SharePct                    float64
dtype: object

In [3]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=rliddb.int.lcog.org,5433;'
                      'Database=GIS_CLMPO;'
                      'Trusted_Connection=yes;')
cursor = conn.cursor()

In [29]:
df['Vehicles.Available'].unique()

array(['No vehicle available', '1 vehicle available',
       '2 vehicles available', '3 or more vehicles available'],
      dtype=object)

In [26]:
df.replace({np.inf: np.nan, -np.inf: np.nan}, inplace=True)

In [27]:
df = df.fillna(0)

In [30]:
# table names: mode_share
# rewrite tables: DROP TABLE mode_share;
# edit accordingly the table
cursor.execute('''
            CREATE TABLE mode_by_vehicles_available(
            year int,
            geo varchar(50),
            mode varchar(50),
            vhcl_avail varchar(50),
            estimate int,
            moe_est int,
            share float,
            moe_share float,
            share_pct float,
            moe_share_pct float
            )
               ''')

In [33]:
df.rename(columns={'Mode.by.Vehicle.Availability': 'Mode', 'Vehicles.Available': 'VHCL_AVAIL'}, inplace=True)

In [36]:
# mode_share
for row in df.itertuples():
    cursor.execute('''
                INSERT INTO mode_by_vehicles_available(year, geo, mode, vhcl_avail, estimate, moe_est, share, moe_share, share_pct, moe_share_pct)
                VALUES (?,?,?,?,?,?,?,?,?,?)
                ''',
                   row.Year,
                   row.Geography,
                   row.Mode,
                   row.VHCL_AVAIL,
                   row.Estimate,
                   row.MOE_Est,
                   row.Share,
                   row.MOE_Share,
                   row.SharePCT,
                   row.MOE_SharePct
                  )
conn.commit()